In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.layers import GaussianNoise, RandomFlip, RandomTranslation, RandomRotation, RandomZoom, RandomContrast
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import cv2
from glob import glob

import matplotlib.pyplot as plt
import time

In [2]:
class dataLoading():
    def __init__(self, X_dir, Y_dir, hyper):
        self.X_dirs = glob(os.path.join(X_dir, '*.png'))
        self.X_dirs.sort()
        self.Y_dirs = glob(os.path.join(Y_dir, '*.png'))
        self.Y_dirs.sort()
        self.w = hyper['Width']
        self.h = hyper['Height']
        self.X_imgs_temp = self.loadImgs(self.X_dirs)
        self.X = self.inputAlign(self.X_imgs_temp)
        del self.X_imgs_temp
        self.Y = self.loadImgs(self.Y_dirs, norm=False, axis=0)
  
        #self.X_train, self.Y_train, self.X_valid, self.Y_valid = train_test_split(self.X_dirs, self.Y_dirs, test_size=0.2)
        
        
        
    def inputAlign(self, imgs):
        
        length = imgs.shape[-1]
        
        X = np.zeros((length, self.h, self.w, 4))
        
        for i in range(length):
            seq = np.zeros((self.h, self.w, 4))

            for j in range(seq.shape[-1]):
                if i - j >= 0:
                    seq[...,j] = imgs[..., i-j]
            X[i,...] = seq
            
        return X
        
        
    def loadImgs(self, img_dirs, w=256, h=256, norm=True, axis=-1):
    
        outputs = None

        for path in img_dirs:
            img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (h,w))
            if norm:
                img = img / 255.0

            if outputs is None:
                if axis == -1:
                    outputs = img.reshape((h,w,1))
                elif axis == 0:
                    outputs = img.reshape((1,h,w,1))
            else:
                if axis == -1:
                    outputs = np.concatenate((outputs, img.reshape(h,w,1)), axis=axis)
                elif axis == 0:
                    outputs = np.concatenate((outputs, img.reshape(1,h,w,1)), axis=axis)

        return outputs

In [3]:
class dataLoaderSingle(tf.data.Dataset):
    def __new__(self, X_dir, Y_dir, hyper):
        self.X_dirs = glob(os.path.join(X_dir, '*.png'))
        self.X_dirs.sort()
        self.Y_dirs = glob(os.path.join(Y_dir, '*.png'))
        self.Y_dirs.sort()
        self.w = hyper['Width']
        self.h = hyper['Height']
        self.X_imgs_temp = self.loadImgs(self, self.X_dirs)
        self.X = self.inputAlign(self, self.X_imgs_temp)
        del self.X_imgs_temp
        self.Y = self.loadImgs(self, self.Y_dirs, norm=False, axis=0)
        
        return tf.data.Dataset.from_tensor_slices({'x': self.X, 'y': self.Y})
        #self.X_train, self.Y_train, self.X_valid, self.Y_valid = train_test_split(self.X_dirs, self.Y_dirs, test_size=0.2)
        
        
        
    def inputAlign(self, imgs):
        
        length = imgs.shape[-1]
        
        X = np.zeros((length, self.h, self.w, 4))
        
        for i in range(length):
            seq = np.zeros((self.h, self.w, 4))

            for j in range(seq.shape[-1]):
                if i - j >= 0:
                    seq[...,j] = imgs[..., i-j]
            X[i,...] = seq
            
        return X
        
        
    def loadImgs(self, img_dirs, w=256, h=256, norm=True, axis=-1):
    
        outputs = None

        for path in img_dirs:
            img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (h,w))
            if norm:
                img = img / 255.0
                
            if outputs is None:
                if axis == -1:
                    outputs = img.reshape((h,w,1))
                elif axis == 0:
                    outputs = img.reshape((1,h,w,1))
            else:
                if axis == -1:
                    outputs = np.concatenate((outputs, img.reshape(h,w,1)), axis=axis)
                elif axis == 0:
                    outputs = np.concatenate((outputs, img.reshape(1,h,w,1)), axis=axis)

        return outputs

In [7]:
x_dir = '/docker/i2i/EntireData/01/set01/fluoroReal'
y_dir = '/docker/i2i/EntireData/01/set01/fluoroAll_DN'
hyper = {'Width':256, 'Height':256}
test = dataLoaderSingle(x_dir, y_dir, hyper)

2021-10-07 06:25:41.120479: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-10-07 06:25:41.477128: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22135 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:01:00.0, compute capability: 8.6


In [8]:
def benchmark(dataset, num_epochs=2):
    start_time = time.perf_counter()
    for epoch_num in range(num_epochs):
        for sample in dataset:
            time.sleep(0.02)
    print("Execution time: ", time.perf_counter() - start_time)

In [10]:
benchmark(test)

Execution time:  5.586222911952063


In [11]:
benchmark(test.prefetch(tf.data.AUTOTUNE))

Execution time:  5.587293624994345


In [12]:
benchmark(tf.data.Dataset.range(2).interleave(lambda _: test))

2021-10-07 06:27:05.630900: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Execution time:  10.993959047016688


In [13]:
benchmark(tf.data.Dataset.range(2).interleave(lambda _: test,
                                              num_parallel_calls=tf.data.AUTOTUNE))

Execution time:  10.982175072014797


In [14]:
data_gen_args = dict(rotation_range=9,
                     width_shift_range=0.16,
                     height_shift_range=0.16,
                     horizontal_flip=True,
                     vertical_flip=True,
                     fill_mode='constant',
                     cval=0,
                     zoom_range=0.1)

In [15]:
data = dataLoading(x_dir, y_dir, hyper)

In [16]:
data.X.shape

(130, 256, 256, 4)

In [17]:
data.Y.shape

(130, 256, 256, 1)

In [18]:
image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen = ImageDataGenerator(**data_gen_args)
seed = 1
image_datagen.fit(data.X, augment=True, seed=seed)
mask_datagen.fit(data.Y, augment=True, seed=seed)

In [19]:
image_gen = image_datagen.flow(data.X, seed=seed)
mask_gen = mask_datagen.flow(data.Y, seed=seed)